# <center>Тема 2. Углубленные приемы pandas, collections, numpy</center>
## <center>Часть 3. Углубленные приемы pandas

In [1]:
import os
import glob
import random
import string
from tqdm.auto import tqdm
from uuid import uuid4

import numpy as np
import pandas as pd

# pd.set_option('mode.chained_assignment', 'warn')

### Индексация

In [2]:
df = pd.DataFrame({'a':[1,2,3,4,5], 
                   'b':[10,20,30,40,50],
                   'c':[100,200,300,400,500]})

In [3]:
df

,a,b,c
0,1,10,100
1,2,20,200
2,3,30,300
3,4,40,400
4,5,50,500


In [4]:
df.iloc[1:4, 1:2]

,b
1,20
2,30
3,40


In [5]:
df.loc[3:4, 'a':'c']

,a,b,c
3,4,40,400
4,5,50,500


In [6]:
df[['a', 'b']][3:]

,a,b
3,4,40
4,5,50


Неправильно

In [7]:
df[df['a'] <= 3]['b'] = (df[df['a'] <= 3]['b'])/10
df #         ^^                     ^^
   #         || dangeous            || dangeous

<ipython-input-7-42a66800d14f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['a'] <= 3]['b'] = (df[df['a'] <= 3]['b'])/10


,a,b,c
0,1,10,100
1,2,20,200
2,3,30,300
3,4,40,400
4,5,50,500


Правильно

In [8]:
df.loc[df['a'] <= 3, 'b'] = df.loc[df['a'] <= 3, 'b'] / 10
df

,a,b,c
0,1,1.0,100
1,2,2.0,200
2,3,3.0,300
3,4,40.0,400
4,5,50.0,500


### inplace

In [9]:
df1 = pd.DataFrame(np.random.rand(100_000, 3))
df1[df1 < 0.5] = np.nan
df2 = pd.DataFrame(np.random.rand(100_000, 3))
df2[df2 < 0.5] = np.nan

In [10]:
df1

,0,1,2
0,NaN,0.948019,NaN
1,NaN,NaN,0.927531
2,0.634315,NaN,0.762433
3,NaN,NaN,NaN
4,0.625133,0.893743,0.787987
...,...,...,...
99995,NaN,NaN,NaN
99996,NaN,0.872125,0.782446
99997,NaN,0.871189,0.505125
99998,0.511679,0.750503,NaN


inplace != faster or more optimized

In [11]:
%%timeit
df = (df1 - df2).dropna()
df = df.rename(columns={col:'d{}'.format(col) for col in df.columns})

5.9 ms ± 621 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
%%timeit
df = (df1 - df2).dropna()
df.rename(columns={col:'d{}'.format(col) for col in df.columns}, inplace=True)

5.12 ms ± 211 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### chaining

In [13]:
new_v = df.dropna().clip(0, 1).sum().mean()
new_v

5.0

### inplace example

In [14]:
df=pd.DataFrame([[1, 2, 8],
                 [3, 4, 8], 
                 [5, 1, 8]], columns=['A', 'B', 'C'])
df

,A,B,C
0,1,2,8
1,3,4,8
2,5,1,8


In [15]:
df.loc[:, 'A'].replace(list(range(10)), 
                       list(map(lambda x: f'number_{x}', range(10))), 
                       inplace=True)
df # data changed - OK

,A,B,C
0,number_1,2,8
1,number_3,4,8
2,number_5,1,8


In [16]:
df.loc[:, ['A', 'B']].replace(list(range(10)), 
                              list(map(lambda x: f'number_{x}', range(10))), 
                              inplace=True)
df # data stays the same! - not OK

,A,B,C
0,number_1,2,8
1,number_3,4,8
2,number_5,1,8


In [17]:
df.loc[:,['A','B']] = df.loc[:,['A','B']].replace(list(range(10)), 
                                                  list(map(lambda x: f'number_{x}', range(10))))
# или 
# df[['A','B']] = df[['A','B']].replace(list(range(10)), 
#                                       list(map(lambda x: f'number_{x}', range(10))))
df

,A,B,C
0,number_1,number_2,8
1,number_3,number_4,8
2,number_5,number_1,8


### Загрузка файлов

In [18]:
def create_frame(n):
    return pd.DataFrame({'id': [str(uuid4()) for _ in range(n)],
                         'date': pd.date_range('2001-01-01', periods=n),
                         'value1': np.random.randint(0, n, size=n),
                         'value2': np.random.randn(n),
                         'value3': np.random.randint(0, n, size=n),
                         'value4': np.random.randn(n)})

In [19]:
os.makedirs('logs', exist_ok=True)
for i in tqdm(range(100)):
    data = create_frame(5000)
    data.to_csv(f'logs/{i:010}.log')

In [20]:
%%timeit

cols = ['id', 'date', 'value1', 'value2', 'value3', 'value4']

# read in loop + append
df = pd.DataFrame(cols)
for file in glob.glob('logs/*'):
    df.append(pd.read_csv(file, usecols=cols))

1.16 s ± 104 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%%timeit

cols = ['id', 'date', 'value1', 'value2', 'value3', 'value4']

# read all + concat
dfs = [pd.read_csv(file, usecols=cols) for file in glob.glob('logs/*')]
pd.concat(dfs, ignore_index=True)

917 ms ± 17.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
for file in glob.glob('logs/*'):
    os.remove(file)
os.removedirs('logs')

### apply

In [23]:
def create_frame(n, n_groups):
    stamps = pd.date_range('20010101', periods=n, freq='ms')
    random.shuffle(stamps.values)    
    return pd.DataFrame({'name': np.random.randint(0,n_groups,size=n),
                         'stamp': stamps,
                         'value': np.random.randint(0,n,size=n),
                         'value2': np.random.randn(n)})


df = create_frame(1000000,10000)

def f_apply(df):
    return df.groupby('name').value2.apply(lambda x: (x-x.mean())/x.std())

def f_unwrap(df):
    g = df.groupby('name').value2
    v = df.value2
    return (v-g.transform(np.mean))/g.transform(np.std)

In [24]:
df.head()

,name,stamp,value,value2
0,3273,2001-01-01 00:14:03.540,441849,0.249259
1,2664,2001-01-01 00:06:27.931,904802,0.854638
2,4807,2001-01-01 00:00:41.650,497979,1.197453
3,5424,2001-01-01 00:08:37.595,703998,0.941025
4,2369,2001-01-01 00:16:34.233,606063,0.416438


In [25]:
%timeit f_apply(df)

3.59 s ± 102 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%timeit f_unwrap(df)

59.4 ms ± 699 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### category

In [27]:
s = pd.Series(np.random.choice(list(string.ascii_letters), 100000))
s.head()

0    f
1    B
2    n
3    b
4    g
dtype: object

In [28]:
'{:0.2f} KB'.format(s.memory_usage(index=False) / 1000)

'800.00 KB'

In [29]:
c = s.astype('category')
'{:0.2f} KB'.format(c.memory_usage(index=False) / 1000)

'102.98 KB'

### memory

dataset link - https://data.world/dataquest/mlb-game-logs

In [30]:
df = pd.read_csv('game_logs.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (12,13,14,15,19,20,81,83,85,87,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [31]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: float64(77), int64(6), object(78)
memory usage: 860.5 MB


In [32]:
object_columns = []
for col in df.columns:
    if df[col].dtype == 'object':
        num_unique_values = len(df[col].unique())
        num_total_values = len(df[col])
        if num_unique_values / num_total_values < 0.5:
            object_columns.append(col)
            df.loc[:,col] = df[col].astype('category')

In [33]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: category(78), float64(77), int64(6)
memory usage: 160.5 MB


In [34]:
for col in df.columns:
    if df[col].dtype == 'int64':
        df.loc[:,col] = df[col].astype('int32')

In [35]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: category(78), float64(77), int32(6)
memory usage: 156.6 MB


In [36]:
for col in df.columns:
    if df[col].dtype == 'float64':
        df.loc[:,col] = df[col].astype('float32')

In [37]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: category(78), float32(77), int32(6)
memory usage: 106.1 MB


In [38]:
column_types = {col: 'category' for col in object_columns}
df = pd.read_csv('game_logs.csv', dtype=column_types)

In [39]:
df.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,na,1,FW1,na,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,na,1,WS3,na,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,18710506,0,Sat,CL1,na,2,RC1,na,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,18710508,0,Mon,CL1,na,3,CH1,na,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,18710509,0,Tue,BS1,na,2,TRO,na,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y


In [40]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: category(78), float64(77), int64(6)
memory usage: 160.5 MB


### merge

In [41]:
df1 = pd.DataFrame({'lkey': ['foo', 'bar', 'baz', 'foo', 'foobar'],
                    'value': [1, 2, 3, 5, 7]})
df2 = pd.DataFrame({'rkey': ['foo', 'bar', 'baz', 'foo', 'foobaz'],
                    'value': [5, 6, 7, 8, 10]})

In [42]:
df1 = df1.set_index('lkey')
df1

,value
lkey,
foo,1
bar,2
baz,3
foo,5
foobar,7


In [43]:
df2 = df2.set_index('rkey')
df2

,value
rkey,
foo,5
bar,6
baz,7
foo,8
foobaz,10


In [44]:
pd.merge(df1, df2, left_index=True, right_index=True)

,value_x,value_y
bar,2,6
baz,3,7
foo,1,5
foo,1,8
foo,5,5
foo,5,8


In [45]:
df1 = df1.reset_index()
df2 = df2.reset_index()

In [46]:
df1.merge(df2, left_on='lkey', right_on='rkey')

,lkey,value_x,rkey,value_y
0,foo,1,foo,5
1,foo,1,foo,8
2,foo,5,foo,5
3,foo,5,foo,8
4,bar,2,bar,6
5,baz,3,baz,7


In [47]:
df1.merge(df2, left_on='lkey', right_on='rkey',
          suffixes=('_left', '_right')) 

,lkey,value_left,rkey,value_right
0,foo,1,foo,5
1,foo,1,foo,8
2,foo,5,foo,5
3,foo,5,foo,8
4,bar,2,bar,6
5,baz,3,baz,7


In [48]:
pd.merge(df1.reset_index(), df2.reset_index(), 
         left_on='lkey', right_on='rkey',
         how='left',
         suffixes=('_left', '_right')) 

,index_left,lkey,value_left,index_right,rkey,value_right
0,0,foo,1,0.0,foo,5.0
1,0,foo,1,3.0,foo,8.0
2,1,bar,2,1.0,bar,6.0
3,2,baz,3,2.0,baz,7.0
4,3,foo,5,0.0,foo,5.0
5,3,foo,5,3.0,foo,8.0
6,4,foobar,7,NaN,NaN,NaN


In [49]:
from pandas.errors import MergeError
try:
    pd.merge(df1.reset_index(), df2.reset_index(), 
         left_on='lkey', right_on='rkey',
         how='left',
         suffixes=('_left', '_right'), validate='many_to_one') 
except MergeError as me:
    print(f'MergeError: {me}')

MergeError: Merge keys are not unique in right dataset; not a many-to-one merge


### resample

In [50]:
df = create_frame(1000000,10000)
df.head()

,name,stamp,value,value2
0,5946,2001-01-01 00:12:49.811,16134,0.359539
1,2735,2001-01-01 00:02:44.340,93373,-1.272354
2,2828,2001-01-01 00:12:14.749,748122,1.031814
3,1836,2001-01-01 00:04:59.155,152797,-1.061109
4,7171,2001-01-01 00:15:12.709,494981,1.333189


In [51]:
for grouper, grouped_df in df.groupby('name'):
    print(grouper)
    display(grouped_df)
    break

0


,name,stamp,value,value2
7757,0,2001-01-01 00:12:25.671,294925,0.699414
10107,0,2001-01-01 00:07:09.155,865621,-0.514566
15075,0,2001-01-01 00:07:15.980,718947,0.911862
37941,0,2001-01-01 00:14:56.537,545141,-0.204749
50293,0,2001-01-01 00:14:09.486,29326,-1.155934
...,...,...,...,...
846546,0,2001-01-01 00:10:11.961,9251,-0.134226
873591,0,2001-01-01 00:10:44.726,404153,-0.586227
880853,0,2001-01-01 00:04:22.235,882363,-1.442294
932396,0,2001-01-01 00:06:47.632,128210,-0.606988


[data offsets](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects)

In [52]:
df.resample('3min', on='stamp').mean()

,name,value,value2
stamp,,,
2001-01-01 00:00:00,4990.544900,500109.742506,0.002223
2001-01-01 00:03:00,5006.775783,500422.579006,0.000783
2001-01-01 00:06:00,4992.482639,500069.579583,0.002915
2001-01-01 00:09:00,5007.501689,499741.783261,-0.001938
2001-01-01 00:12:00,4984.731250,500698.123244,-0.000673
2001-01-01 00:15:00,5011.371110,500206.227390,0.000439


In [53]:
df.set_index('stamp').groupby(['name', pd.Grouper(freq='3min')]).mean().head(15)

value    value2
name stamp                                       
0    2001-01-01 00:00:00  512490.235294 -0.385332
     2001-01-01 00:03:00  462403.230769  0.155350
     2001-01-01 00:06:00  604510.375000  0.064935
     2001-01-01 00:09:00  435976.333333  0.460261
     2001-01-01 00:12:00  524732.900000  0.130077
     2001-01-01 00:15:00  503288.125000 -0.451939
1    2001-01-01 00:00:00  320859.357143 -0.433724
     2001-01-01 00:03:00  480367.125000 -0.367922
     2001-01-01 00:06:00  540401.043478  0.170424
     2001-01-01 00:09:00  463250.458333  0.064735
     2001-01-01 00:12:00  512856.529412 -0.052214
     2001-01-01 00:15:00  555214.000000 -0.032377
2    2001-01-01 00:00:00  562533.312500 -0.139289
     2001-01-01 00:03:00  524169.187500 -0.398029
     2001-01-01 00:06:00  644169.000000  0.110516

In [54]:
df.groupby('name').resample('3min', on='stamp').mean().head(15)

name          value    value2
name stamp                                             
0    2001-01-01 00:00:00   0.0  512490.235294 -0.385332
     2001-01-01 00:03:00   0.0  462403.230769  0.155350
     2001-01-01 00:06:00   0.0  604510.375000  0.064935
     2001-01-01 00:09:00   0.0  435976.333333  0.460261
     2001-01-01 00:12:00   0.0  524732.900000  0.130077
     2001-01-01 00:15:00   0.0  503288.125000 -0.451939
1    2001-01-01 00:00:00   1.0  320859.357143 -0.433724
     2001-01-01 00:03:00   1.0  480367.125000 -0.367922
     2001-01-01 00:06:00   1.0  540401.043478  0.170424
     2001-01-01 00:09:00   1.0  463250.458333  0.064735
     2001-01-01 00:12:00   1.0  512856.529412 -0.052214
     2001-01-01 00:15:00   1.0  555214.000000 -0.032377
2    2001-01-01 00:00:00   2.0  562533.312500 -0.139289
     2001-01-01 00:03:00   2.0  524169.187500 -0.398029
     2001-01-01 00:06:00   2.0  644169.000000  0.110516